In [4]:
import pandas as pd
from keplergl import KeplerGl
import geopandas as gpd
# https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Current-Year-To-Date-/5uac-w243
small_file = 'data/NYPD_Complaint_Data_Current__Year_To_Date_.csv'
df = gpd.read_file(small_file)
# df.head(100)

In [5]:
df = df[df.Longitude != ""]
df.Longitude = df.Longitude.astype(float)
df.Latitude = df.Latitude.astype(float)
df.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,...,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,geometry
0,314773184,48,BRONX,12/31/2019,18:00:00,,,COMPLETED,,,...,,45-64,WHITE HISPANIC,F,1016990,244612,40.838026,-73.881681,"(40.838026269000075, -73.88168118799997)",None
1,289837961,25,MANHATTAN,12/30/2019,20:30:00,12/31/2019,10:00:00,COMPLETED,,,...,,25-44,WHITE HISPANIC,F,999296,230862,40.800334,-73.945657,"(40.800334261000046, -73.94565697199994)",None
2,535744284,77,BROOKLYN,12/24/2019,16:55:00,12/24/2019,17:00:00,COMPLETED,,,...,,18-24,WHITE,M,1001068,183317,40.669832,-73.939376,"(40.66983179600004, -73.93937555099996)",None
3,895678119,52,BRONX,12/30/2019,19:32:00,,,COMPLETED,,,...,,UNKNOWN,UNKNOWN,D,1009690,257590,40.873671,-73.908014,"(40.87367103500002, -73.90801364899994)",None
4,299841674,18,MANHATTAN,12/30/2019,15:30:00,12/30/2019,16:50:00,COMPLETED,,,...,,25-44,WHITE HISPANIC,M,985766,216503,40.760935,-73.994529,"(40.76093528000007, -73.99452906599998)",None


In [6]:
df.columns

Index(['CMPLNT_NUM', 'ADDR_PCT_CD', 'BORO_NM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM',
       'CMPLNT_TO_DT', 'CMPLNT_TO_TM', 'CRM_ATPT_CPTD_CD', 'HADEVELOPT',
       'HOUSING_PSA', 'JURISDICTION_CODE', 'JURIS_DESC', 'KY_CD', 'LAW_CAT_CD',
       'LOC_OF_OCCUR_DESC', 'OFNS_DESC', 'PARKS_NM', 'PATROL_BORO', 'PD_CD',
       'PD_DESC', 'PREM_TYP_DESC', 'RPT_DT', 'STATION_NAME', 'SUSP_AGE_GROUP',
       'SUSP_RACE', 'SUSP_SEX', 'TRANSIT_DISTRICT', 'VIC_AGE_GROUP',
       'VIC_RACE', 'VIC_SEX', 'X_COORD_CD', 'Y_COORD_CD', 'Latitude',
       'Longitude', 'Lat_Lon', 'geometry'],
      dtype='object')

In [7]:
#Create a basemap 
map = KeplerGl(height=600, width=800)#show the map
map

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(height=600)

In [8]:
# Create a gepdataframe
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))# Add data to Kepler
map.add_data(data=gdf, name="crimes")

In [9]:
def count_incidents_neighborhood(data, neighb):
 # spatial join and group by to get count of incidents in each poneighbourhood 
 joined = gpd.sjoin(gdf, neighb, op="within")
 grouped = joined.groupby('ntaname').size()
 df = grouped.to_frame().reset_index()
 df.columns = ['ntaname', 'count']
 merged = Neighborhood.merge(df, on='ntaname', how='outer')
 merged['count'].fillna(0,inplace=True)
 merged['count'] = merged['count'].astype(int)
 return merged

merged_gdf = count_incidents_neighborhood(gdf, Neighborhood)

In [ ]:
map2 = KeplerGl(height=600, width=800)
# Add data to Kepler
map2.add_data(data=merged_gdf, name=”NeighborhoodCrimes”)
map2

In [1]:
import pandas as pd
from keplergl import KeplerGl
import geopandas as gpd
# https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Current-Year-To-Date-/5uac-w243
small_file = 'data/nyct2010.csv'
df = gpd.read_file(small_file)

In [2]:
df.head()

,BoroCode,the_geom,CTLabel,BoroName,CT2010,BoroCT2010,CDEligibil,NTACode,NTAName,PUMA,Shape_Leng,Shape_Area,geometry
0,5,MULTIPOLYGON (((-74.07920577013245 40.64343078...,9,Staten Island,000900,5000900,E,SI22,West New Brighton-New Brighton-St. George,3903,7729.01679383,2497009.71359,None
1,1,MULTIPOLYGON (((-73.96432543478758 40.75638153...,98,Manhattan,009800,1009800,I,MN19,Turtle Bay-East Midtown,3808,5534.19981063,1906016.35002,None
2,1,MULTIPOLYGON (((-73.96802436915851 40.75957814...,100,Manhattan,010000,1010000,I,MN19,Turtle Bay-East Midtown,3808,5692.16873705,1860938.37721,None
3,1,MULTIPOLYGON (((-73.97124277307127 40.76093641...,102,Manhattan,010200,1010200,I,MN17,Midtown-Midtown South,3807,5687.80243866,1860992.70485,None
4,1,MULTIPOLYGON (((-73.97445730550224 40.76229308...,104,Manhattan,010400,1010400,I,MN17,Midtown-Midtown South,3807,5693.03636655,1864600.37503,None


In [3]:
# Create a gepdataframe
gdf = gpd.GeoDataFrame(df, geometry=df.the_geom)# Add data to Kepler
map.add_data(data=gdf, name="crimes")

TypeError: Input must be valid geometry objects: MULTIPOLYGON (((-74.07920577013245 40.64343078374567, -74.07913504647637 40.64333789954809, -74.07819773831595 40.64211481672276, -74.07884632409665 40.641995568528166, -74.0791223401088 40.64194431928626, -74.07965266950632 40.641855351552806, -74.08055489346589 40.64169150377349, -74.08148001617707 40.64155046162583, -74.08130714145031 40.64074191709323, -74.08110049694088 40.639922825339106, -74.08191207797016 40.6398059117602, -74.08194326513642 40.63993327788856, -74.0820464807701 40.640338526958104, -74.08228199570526 40.640600896453016, -74.08503312179074 40.64019785806871, -74.0872057798552 40.639895367647135, -74.08709163679771 40.64033437636213, -74.08701899114665 40.640613790850445, -74.08682767812078 40.64133802397936, -74.08667746827268 40.64190963327233, -74.08652380457303 40.64256408357222, -74.08575006381034 40.642846120676055, -74.08496857468032 40.643130075816266, -74.08414352403344 40.64343089730628, -74.0820726845645 